## ChatGPT来了，更快的速度更低的价格

In [1]:
import openai
import os

openai.api_key = os.environ.get("OPENAI_API_KEY")

class Conversation:
    def __init__(self, prompt, num_of_round):
        self.prompt = prompt
        self.num_of_round = num_of_round
        self.messages = []
        self.messages.append({"role": "system", "content": self.prompt})

    def ask(self, question):
        try:
            self.messages.append( {"role": "user", "content": question})
            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=self.messages,
                temperature=0.5,
                max_tokens=2048,
                top_p=1,
            )
        except Exception as e:
            print(e)
            return e

        message = response["choices"][0]["message"]["content"]
        self.messages.append({"role": "assistant", "content": message})
        
        if len(self.messages) > self.num_of_round*2 + 1:
            del self.messages[1:3]
        return message


In [2]:
prompt = """你是一个中国厨师，用中文回答做菜的问题。你的回答需要满足以下要求:
1. 你的回答必须是中文
2. 回答限制在100个字以内"""
conv1 = Conversation(prompt, 3)
question1 = "你是谁？"
print("User : %s" % question1)
print("Assistant : %s\n" % conv1.ask(question1))

question2 = "请问鱼香肉丝怎么做？"
print("User : %s" % question2)
print("Assistant : %s\n" % conv1.ask(question2))

question3 = "那蚝油牛肉呢？"
print("User : %s" % question3)
print("Assistant : %s\n" % conv1.ask(question3))


User : 你是谁？
Assistant : 我是一个AI语言模型，可以用中文回答各种问题，包括做菜的问题。

User : 请问鱼香肉丝怎么做？
Assistant : 鱼香肉丝的做法：
1. 猪里脊肉切成细丝，用料酒、盐、生抽腌制10分钟。
2. 香葱、姜、蒜切末，泡辣椒切段备用。
3. 锅里放油，将腌制好的猪肉丝煸炒至变色，盛出备用。
4. 锅中留底油，加入泡辣椒段、葱姜蒜末煸炒出香味，加入豆瓣酱、甜面酱、醋、糖、盐、鸡精、清水调成酱汁。
5. 将煸炒好的猪肉丝倒入酱汁中，翻炒均匀，最后加入青红椒丝翻炒即可。

User : 那蚝油牛肉呢？
Assistant : 蚝油牛肉的做法：
1. 将牛肉切成薄片，用料酒、盐、生粉腌制10分钟。
2. 青红椒切丝，葱姜切末备用。
3. 锅中倒油，将腌制好的牛肉片煸炒至变色，盛出备用。
4. 锅中留底油，加入葱姜末煸炒出香味，加入青红椒丝煸炒。
5. 加入蚝油、盐、糖、鸡精、清水调成酱汁，煮沸后加入煸炒好的牛肉片，翻炒均匀。
6. 最后加入少量水淀粉勾芡，翻炒均匀即可。



In [9]:
question4 = "我问你的第一个问题是什么？"
print("User : %s" % question4)
print("Assistant : %s\n" % conv1.ask(question4))


User : 我问你的第一个问题是什么？
Assistant : 你的第一个问题是“你是谁？”



In [11]:
question5 = "我问你的第一个问题是什么？"
print("User : %s" % question5)
print("Assistant : %s\n" % conv1.ask(question5))

User : 我问你的第一个问题是什么？
Assistant : 你的第一个问题是“请问鱼香肉丝怎么做？”



### 通过API计算Token数量

In [12]:
class Conversation2:
    def __init__(self, prompt, num_of_round):
        self.prompt = prompt
        self.num_of_round = num_of_round
        self.messages = []
        self.messages.append({"role": "system", "content": self.prompt})

    def ask(self, question):
        try:
            self.messages.append( {"role": "user", "content": question})
            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=self.messages,
                temperature=0.5,
                max_tokens=2048,
                top_p=1,
            )
        except Exception as e:
            print(e)
            return e

        message = response["choices"][0]["message"]["content"]
        num_of_tokens = response['usage']['total_tokens']
        self.messages.append({"role": "assistant", "content": message})
        
        if len(self.messages) > self.num_of_round*2 + 1:
            del self.messages[1:3]
        return message, num_of_tokens

In [13]:
conv2 = Conversation2(prompt, 3)
questions = [question1, question2, question3, question4, question5]
for question in questions:
    answer, num_of_tokens = conv2.ask(question)
    print("询问 {%s} 消耗的token数量是 : %d" % (question, num_of_tokens))

询问 {你是谁？} 消耗的token数量是 : 121
询问 {请问鱼香肉丝怎么做？} 消耗的token数量是 : 419
询问 {那蚝油牛肉呢？} 消耗的token数量是 : 731
Rate limit reached for default-gpt-3.5-turbo in organization org-QJFEqyGGulP668cGrGtNfsUU on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method.


TypeError: cannot unpack non-iterable RateLimitError object

### 通过Tiktoken库计算Token数量

In [14]:
import tiktoken
encoding = tiktoken.get_encoding("cl100k_base")

conv2 = Conversation2(prompt, 3)
question1 = "你是谁？"
answer1, num_of_tokens = conv2.ask(question1)
print("总共消耗的token数量是 : %d" % (num_of_tokens))

prompt_count = len(encoding.encode(prompt))
question1_count = len(encoding.encode(question1))
answer1_count = len(encoding.encode(answer1))
total_count = prompt_count + question1_count + answer1_count
print("Prompt消耗 %d Token, 问题消耗 %d Token，回答消耗 %d Token，总共消耗 %d Token" % (prompt_count, question1_count, answer1_count, total_count))

总共消耗的token数量是 : 107
Prompt消耗 65 Token, 问题消耗 5 Token，回答消耗 24 Token，总共消耗 94 Token


In [15]:
system_start_count = len(encoding.encode("<|im_start|>system\n"))
print(encoding.encode("<|im_start|>system\n"))
end_count = len(encoding.encode("<|im_end|>\n"))
print(encoding.encode("<|im_end|>\n"))
user_start_count = len(encoding.encode("<|im_start|>user\n"))
print(encoding.encode("<|im_start|>user\n"))
assistant_start_count = len(encoding.encode("<|im_start|>assistant\n"))
print(encoding.encode("<|im_start|>assistant\n"))

total_mark_count = system_start_count + user_start_count + assistant_start_count + end_count*2
print("系统拼接的标记消耗 %d Token" % total_mark_count)


[27, 91, 318, 5011, 91, 29, 9125, 198]
[27, 91, 318, 6345, 91, 397]
[27, 91, 318, 5011, 91, 29, 882, 198]
[27, 91, 318, 5011, 91, 29, 78191, 198]
系统拼接的标记消耗 36 Token


## Gradio帮你快速搭建一个聊天界面

In [16]:
%conda install gradio

Solving environment: done

## Package Plan ##

  environment location: /Users/xiaodong/opt/anaconda3

  added / updated specs:
    - gradio


The following packages will be UPDATED:

  ca-certificates    conda-forge::ca-certificates-2022.12.~ --> anaconda/pkgs/main::ca-certificates-2023.01.10-hecd8cb5_0 

The following packages will be SUPERSEDED by a higher-priority channel:

  certifi            conda-forge/noarch::certifi-2022.12.7~ --> anaconda/pkgs/main/osx-64::certifi-2022.12.7-py38hecd8cb5_0 
  openssl            conda-forge::openssl-1.1.1t-hfd90126_0 --> anaconda/pkgs/main::openssl-1.1.1t-hca72f7f_0 




Preparing transaction: done
Verifying transaction: done
Executing transaction: done

Note: you may need to restart the kernel to use updated packages.


In [17]:
import gradio as gr
prompt = """你是一个中国厨师，用中文回答做菜的问题。你的回答需要满足以下要求:
1. 你的回答必须是中文
2. 回答限制在100个字以内"""

conv = Conversation(prompt, 5)

def predict(input, history=[]):
    history.append(input)
    response = conv.ask(input)
    history.append(response)
    responses = [(u,b) for u,b in zip(history[::2], history[1::2])]
    return responses, history

with gr.Blocks(css="#chatbot{height:350px} .overflow-y-auto{height:500px}") as demo:
    chatbot = gr.Chatbot(elem_id="chatbot")
    state = gr.State([])

    with gr.Row():
        txt = gr.Textbox(show_label=False, placeholder="Enter text and press enter").style(container=False)

    txt.submit(predict, [txt, state], [chatbot, state])

demo.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
